<a href="https://colab.research.google.com/github/Sheepybloke2-0/CSE847-Project/blob/main/Model_Training/tinyml_seedling_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check that imports for the rest of the file work.
import tensorflow as tf

!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs
!pip install -q kaggle
! pip install -q tensorflow-model-optimization

!apt-get update && apt-get -qq install xxd

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Allow matplotlib images to render immediately.
%matplotlib inline

import glob
import imageio
import os
import PIL
from tensorflow.keras import layers
import time
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot
import tempfile

from IPython import display

     |████████████████████████████████| 1.0MB 17.3MB/s 
     |████████████████████████████████| 174kB 19.2MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,402 kB]
Get:11 http://security.ubuntu.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
from google.colab import files
import pathlib
kaggleJson = pathlib.Path("/content/.kaggle/kaggle.json")
if not kaggleJson.is_file():
  # files.upload()
  !mkdir -p /content/.kaggle/
  %cd /content/gdrive/My\ Drive/
  !cp ./kaggle.json /content/.kaggle/
  !chmod 600 /content/.kaggle/kaggle.json
%env KAGGLE_CONFIG_DIR=/content/.kaggle
!echo $KAGGLE_CONFIG_DIR
!kaggle config view
!kaggle datasets list

/content/gdrive/My Drive
env: KAGGLE_CONFIG_DIR=/content/.kaggle
/content/.kaggle
Configuration values from /content/.kaggle
- username: reubenlewis
- path: None
- proxy: None
- competition: None
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              222KB  2021-04-05 06:19:09           1790  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44           1171  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            901  
imsparsh/musicnet-dataset         

In [4]:
# NOTE: Remove the zip file and redownload if the unzip isn't working properly
imageZip = pathlib.Path('/content/gdrive/My Drive/v2-plant-seedlings-dataset.zip')
if not imageZip.is_file():
  !kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset
  !ls
else:
  %cd /content/gdrive/My\ Drive/
!unzip v2-plant-seedlings-dataset.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/nonsegmentedv2/Charlock/308.png  
  inflating: /content/nonsegmentedv2/Charlock/309.png  
  inflating: /content/nonsegmentedv2/Charlock/31.png  
  inflating: /content/nonsegmentedv2/Charlock/310.png  
  inflating: /content/nonsegmentedv2/Charlock/311.png  
  inflating: /content/nonsegmentedv2/Charlock/312.png  
  inflating: /content/nonsegmentedv2/Charlock/313.png  
  inflating: /content/nonsegmentedv2/Charlock/314.png  
  inflating: /content/nonsegmentedv2/Charlock/315.png  
  inflating: /content/nonsegmentedv2/Charlock/316.png  
  inflating: /content/nonsegmentedv2/Charlock/317.png  
  inflating: /content/nonsegmentedv2/Charlock/318.png  
  inflating: /content/nonsegmentedv2/Charlock/319.png  
  inflating: /content/nonsegmentedv2/Charlock/32.png  
  inflating: /content/nonsegmentedv2/Charlock/320.png  
  inflating: /content/nonsegmentedv2/Charlock/321.png  
  inflating: /content/nonsegmentedv2/Charlock/322.png  

In [5]:
rootDir = pathlib.Path('/content/')
%cd /content/
!rm -r sample_data

imageCount = len(list(rootDir.glob('nonsegmentedv2/*/*.png')))
print(imageCount)

/content
5539


In [6]:
# CONSTANTS GO HERE
BUFFER_SIZE = imageCount
BATCH_SIZE = 256
IM_SIZE = (128, 128)
IM_DIMS = (128, 128,3)
EPOCHS = 100
FULL_MODEL = '/content/gdrive/My Drive/full_model'
LITE_MODEL = '/content/gdrive/My Drive/lite_model.tflite'
LITE_MODEL_ESC = '/content/gdrive/My\ Drive/lite_model.tflite'
QUANT_MODEL = '/content/gdrive/My Drive/quant_model.tflite'
QUANT_MODEL_ESC = '/content/gdrive/My\ Drive/quant_model.tflite'
MICRO_MODEL = '/content/gdrive/My Drive/micro_model.cc'
MICRO_MODEL_ESC = '/content/gdrive/My\ Drive/micro_model.cc'

In [7]:
trainPath = os.path.join(rootDir, 'nonsegmentedv2')

trainDataGen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        rotation_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.2)

testDataGen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

trainGenerator = trainDataGen.flow_from_directory(
    trainPath,
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE,
    subset='training'
)

testGenerator = testDataGen.flow_from_directory(
    trainPath,
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation'
)


Found 4437 images belonging to 12 classes.
Found 1102 images belonging to 12 classes.


In [8]:
def classificationModel(numClasses):
  L2_lambda = 0.01

  model = tf.keras.Sequential()
  model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=IM_DIMS, kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D((2,2)))

  model.add(layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D((2,2)))

  model.add(layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))
  model.add(layers.BatchNormalization())

  model.add(layers.Dropout(0.2))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))
  model.add(layers.Dense(numClasses, kernel_regularizer=tf.keras.regularizers.l2(L2_lambda)))

  model.summary()
  return model

In [9]:
model = classificationModel(trainGenerator.num_classes)

schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.0001, 100000, 0.98)
opt = tf.keras.optimizers.Adam(learning_rate=schedule)

model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(
    trainGenerator, 
    validation_data=testGenerator, 
    epochs=EPOCHS,
    steps_per_epoch=trainGenerator.n//trainGenerator.batch_size,
    validation_steps=testGenerator.n//testGenerator.batch_size
    )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        1

In [11]:
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: /tmp/tmpe0j_luij.h5


In [12]:
loaded_model = tf.keras.models.load_model(keras_file)

# Finish pruning after 5 epochs
pruning_epochs = 100
end_step = np.ceil(4432 / BATCH_SIZE).astype(np.int32) * pruning_epochs

# Define pruning configuration
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step,
                                                               frequency=100)
}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
model_for_pruning.summary()

# Recompile the model
model_for_pruning.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 126, 126, 16)      882       
_________________________________________________________________
prune_low_magnitude_batch_no (None, 126, 126, 16)      65        
_________________________________________________________________
prune_low_magnitude_max_pool (None, 63, 63, 16)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 61, 61, 32)        9250      
_________________________________________________________________
prune_low_magnitude_batch_no (None, 61, 61, 32)        129       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 30, 30, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 28, 28, 64)        3

In [13]:
# Model callbacks
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

# Fitting data
model_for_pruning.fit(trainGenerator, 
                      validation_data=testGenerator,
                      steps_per_epoch=trainGenerator.n//trainGenerator.batch_size,
                      validation_steps=testGenerator.n//testGenerator.batch_size,
                      batch_size=BATCH_SIZE,
                      epochs=pruning_epochs,
                      verbose=1,
                      callbacks=callbacks)

Epoch 1/5
17/17 [==============================] - 54s 3s/step - loss: 8.8683 - accuracy: 0.5193 - val_loss: 10.6756 - val_accuracy: 0.1084
Epoch 2/5
17/17 [==============================] - 49s 3s/step - loss: 10.1814 - accuracy: 0.2262 - val_loss: 8.6738 - val_accuracy: 0.1396
Epoch 3/5
17/17 [==============================] - 49s 3s/step - loss: 9.9659 - accuracy: 0.1650 - val_loss: 9.9392 - val_accuracy: 0.0977
Epoch 4/5
17/17 [==============================] - 49s 3s/step - loss: 9.3396 - accuracy: 0.1553 - val_loss: 12.2462 - val_accuracy: 0.0771
Epoch 5/5
17/17 [==============================] - 49s 3s/step - loss: 11.8610 - accuracy: 0.1513 - val_loss: 11.8647 - val_accuracy: 0.0693


In [14]:
score = model_for_pruning.evaluate(testGenerator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 11.913203239440918
Test accuracy: 0.06987296044826508


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

print("final model")
model_for_export.summary()

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved baseline model to:', pruned_keras_file)

percentages = tf.keras.models.load_model(pruned_keras_file)
for i, w in enumerate(percentages.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            percentages.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

final model
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization_16 (Batc (None, 126, 126, 16)      64        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 61, 32)        4640      
_________________________________________________________________
batch_normalization_17 (Batc (None, 61, 61, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28

In [ ]:
import zipfile
_, zip3 = tempfile.mkstemp(".zip")
with zipfile.ZipFile(zip3, "w", compression=zipfile.ZIP_DEFLATED) as f:
    f.write(pruned_keras_file)
print(
    "Size of the pruned model before compression: %.2f Mb"
    % (os.path.getsize(pruned_keras_file) / float(2 ** 20))
)
print(
    "Size of the pruned model after compression: %.2f Mb"
    % (os.path.getsize(zip3) / float(2 ** 20))
)

Size of the pruned model before compression: 42.68 Mb
Size of the pruned model after compression: 24.84 Mb


In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
model_no_quant_tflite = converter.convert()

# Now test with quantazation for int8: 
# https://stackoverflow.com/questions/65902185/tfliteconverter-representative-dataset-from-keras-preprocessing-image-dataset-fr
def representative_dataset():
  for _ in range(10):
    img = trainGenerator.next()
    yield [img[0]]

converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
model_quant_tflite = converter.convert()

# Save the model to disk
open(LITE_MODEL, "wb").write(model_no_quant_tflite)
open(QUANT_MODEL, "wb").write(model_quant_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpqn4y_z30/assets


INFO:tensorflow:Assets written to: /tmp/tmpqn4y_z30/assets


INFO:tensorflow:Assets written to: /tmp/tmpnxq5u5fi/assets


INFO:tensorflow:Assets written to: /tmp/tmpnxq5u5fi/assets


11192472

In [ ]:
# TODO: Add some testing to compare the models

In [ ]:
# Check the size of the model
# Calculate size
size_tf = os.path.getsize(FULL_MODEL)
size_tflite = os.path.getsize(LITE_MODEL)
size_quant = os.path.getsize(QUANT_MODEL)

pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_tflite} bytes ", f"(reduced by {size_tf - size_tflite} bytes)"],
     ["TensorFlow Quant Lite", f"{size_quant} bytes ", f"(reduced by {size_tflite - size_quant} bytes)"]],
     columns=["Model", "Size", ""], index="Model")

,Size,
Model,,
TensorFlow,4096 bytes,
TensorFlow Lite,44707136 bytes,(reduced by -44703040 bytes)
TensorFlow Quant Lite,11192472 bytes,(reduced by 33514664 bytes)


In [ ]:
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {QUANT_MODEL_ESC} > {MICRO_MODEL_ESC}
# Update variable names
REPLACE_TEXT = QUANT_MODEL.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MICRO_MODEL}

sed: -e expression #1, char 20: unterminated `s' command


In [ ]:
# TODO: Write and deploy the model